In [ ]:
# Smart Home Automation using AI - Google Colab Notebook

# --- Step 1: Setup ---

# Install Required Libraries
!pip install tensorflow
!pip install torch torchvision
!pip install opencv-python
!pip install matplotlib seaborn
!pip install scikit-learn
!pip install SpeechRecognition pyaudio flask

# --- Step 2: Imports ---

import tensorflow as tf
import torch
import torchvision
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import random
import speech_recognition as sr
from flask import Flask, render_template, request, jsonify

# --- Step 3: Project Introduction ---

"""
## Smart Home Automation Using AI
This project simulates an AI-driven smart home system capable of:
- Predicting user activities for automation (e.g., lights ON)
- Detecting room activity via camera input
- Accepting voice commands for smart control
- Hosting a simple web dashboard using Flask
"""

# --- Step 4: Simulated Dataset Creation (Sensor Data) ---

def generate_sample():
    return [random.randint(0, 1) for _ in range(10)]

def generate_dataset(n=1000):
    X = [generate_sample() for _ in range(n)]
    y = [1 if sum(sample) > 5 else 0 for sample in X]  # Active scenario
    return np.array(X), np.array(y)

X, y = generate_dataset()
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# --- Step 5: LSTM Model for Activity Prediction ---

model = Sequential()
model.add(LSTM(64, input_shape=(10, 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

x_train_lstm = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test_lstm = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

history = model.fit(x_train_lstm, y_train, epochs=10, validation_data=(x_test_lstm, y_test))

# --- Step 6: Evaluation ---

loss, accuracy = model.evaluate(x_test_lstm, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

y_pred = (model.predict(x_test_lstm) > 0.5).astype(int)
print(classification_report(y_test, y_pred))

# --- Step 7: Computer Vision - Room Detection ---

def detect_edges(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)
    plt.imshow(edges, cmap='gray')
    plt.title("Edge Detection - Room")
    plt.axis('off')
    plt.show()

# Example (commented for now)
# detect_edges('room_sample.jpg')

# --- Step 8: Voice Command Module ---

def listen_command():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            print(f"Command: {text}")
            return text.lower()
        except sr.UnknownValueError:
            print("Could not understand audio")
        except sr.RequestError as e:
            print(f"Could not request results: {e}")

# Example (run only in interactive environments)
# listen_command()

# --- Step 9: Flask Web Dashboard (Basic API) ---

app = Flask(__name__)

@app.route('/')
def index():
    return "<h2>Smart Home AI Dashboard</h2><p>Status: Online</p>"

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json['sensors']  # 10 sensor values
    data = np.array(data).reshape(1, 10, 1)
    prediction = model.predict(data)[0][0]
    return jsonify({"activity": "active" if prediction > 0.5 else "inactive", "score": float(prediction)})

# Uncomment to run Flask server locally
# if __name__ == '__main__':
#     app.run(debug=True)

# --- End of Notebook ---
